In [ ]:
import itertools as it
import os

os.environ["PRIVBAYES_BIN"] = "./ydnpd/synthesis/privbayes"

import ray
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Markdown

import ydnpd

## Tasks

In [ ]:
total_task_size = 0
for idx, task in enumerate(ydnpd.span_hparam_tasks(), start=1):
    print(f"{idx:2} {task}")
    total_task_size += task.size()
print(f"\nTotal task size: {total_task_size}")

## Execute Tasks

In [ ]:
ray.init(num_cpus=3)

In [ ]:
hparams_tasks = ydnpd.span_hparam_ray_tasks(
    with_wandb=False
)
hparam_tasks_results = ray.get(hparams_tasks)
hparam_tasks_results = list(it.chain(*hparam_tasks_results))

## Utility-Related Tasks

In [ ]:
for metric in ydnpd.EVALUATION_METRICS:
    display(Markdown(f"## {metric}"))

    g1, g2 = ydnpd.UtilityTask.plot(
        hparam_tasks_results, ydnpd.config.EXPERIMENTS, metric=metric
    )
    display(g1.fig)
    plt.close(g1.fig)

    display(g2.fig)
    plt.close(g2.fig)

In [ ]:
for metric in ydnpd.EVALUATION_METRICS:

    display(Markdown(f"## {metric}"))
    with pd.option_context("display.max_rows", None):
        display(
            ydnpd.UtilityTask.evaluate(
                hparam_tasks_results, ydnpd.config.EXPERIMENTS, metric
            )
            .multiply(100)
            .round(1)
        )